# Your First Locally Hosted Sentinel Notebook

The purpose of this notebook is to ensure your local Sentinenl Notebook environment is set up correctly, additionally, it enables you to connect your local notebook to Microsoft Sentinel and ensures this works correctly as well. Have fun!

## 1. Importing msticpy and checking the version

In [ ]:
import msticpy as mp

mp.__version__

## 2. Ensure the environments are correct & sanity check

This section ensures that the kernel sellected is running on the virtual environment you created that has all the necessary libraries. 

In [ ]:
import sys
print(sys.executable)
print(mp.__file__)
print(mp.__version__)


The command below ensures that the `msticpy` library is imported and runs correctly. Just a sanity check :) 

In [ ]:
help(mp)

### 3. Initialising & setting up the `msticpyconfig.yaml' file

On running this cell you will yill get a GUI through which you can set up your `msticpyconfig.yaml` which essentially connects your notebook to Sentinel and any other tools you plan to use. 

The bare minimum settings you need to set up are Microsoft Sentinel for your workspace details and Azure settings. For testing purposes I'd highly recommend using `interactive` as your `auth_method` once you get the hang of it you can set up a dedeicated managed identity. 

Once you save the settings, they will be stored in your current working directory. Any notebook you start from this diretory will inherit your config settings. 

In [ ]:
mp.init_notebook(namespace=globals())

from msticpy.config import MpConfigFile, MpConfigEdit

# create a config object and load any defaults
mpconfig = MpConfigFile()
mpconfig.load_default()   # this will show empty user settings if you have none yet

# launch the config editor UI
mpedit = MpConfigEdit(settings=mpconfig)
mpedit

## Test your Notebook against your Sentinel Data

The steps below are derived from [A Getting Started Guide For Azure Sentinel ML Notebooks](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/A%20Getting%20Started%20Guide%20For%20Azure%20Sentinel%20ML%20Notebooks.ipynb)

You might get a pop up window requesting you to authenticate your Microsoft acount, this is normal

### 4. Load a QueryProvider for Microsoft Sentinel

To start, we are going to load up a `QueryProvider`
for Microsoft Sentinel. The `QueryProvider` is the object you use to
querying data from MS Sentinel and make it available to view and analyze in the notebook.
There are two steps to do this:
1. Create the `QueryProvider`
2. run the `connect` function to authenticate to the Sentinel workspace.

<div style="border: solid; padding: 5pt"><b>Note:</b>
If you see a warning "Runtime dependency of PyGObject is missing" when loading the<br>
Microsoft Sentinel driver, please see the FAQ section at the end of this notebook.<br>
The warning does not impact any functionality of the notebooks.
</div>
<br>
<details>
    <summary>More about query providers...</summary>
<p>Query results are always returned as <i>pandas</i> DataFrames.</p>
<p>If you are new
to using <i>pandas</i> look at the <b>Introduction to Pandas</b> section at in
the <b>A Tour of Cybersec notebook features</b> notebook.</p>
<br>
<p>
The query provider supports other data sources, as well as Microsoft Sentinel.</p>
<p>
Other data sources supported by the `QueryProvider` class include Microsoft Defender for Endpoint,
Splunk, Microsoft Graph API, Azure Resource Graph but these are not covered here.
</p>
<p>
Most query providers come with a range of built-in queries
for common data operations. You can also a query provider to run custom queries against
Microsoft Sentinel data.
</p>
<br>
Once you've loaded a QueryProvider you'll normally need to authenticate
to the data source (in this case Microsoft Sentinel).
<ul>
<li><a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProviders.html#instantiating-a-query-provider >MSTICPy Documentation</a>.</li>
</ul>
</details>
<br>


In [6]:
# Refresh any config items that might have been saved
# to the msticpyconfig in the previous steps.
mp.settings.refresh_config()

# Initialize a QueryProvider for Microsoft Sentinel
qry_prov = mp.QueryProvider("AzureSentinel")

## 5. Authenticate to the Microsoft Sentinel workspace

Next we need to authenticate.

The code cell immediately following this section will start the authentication process.

In Azure ML notebooks the authentication will default to using the credentials
you used to authentication to the Azure ML workspace.

More information:

<details>
   <summary>Alternative authentication options</summary>
Instead of using the Azure ML credentials, you can opt to use
one of the following:
<ul>
    <li>Device authentication</li>
    <li>Azure CLI credentials</li>
</ul>

<p><b>Device authentication</b> uses a unique code generated on your client
as an additional authentication factor. When prompted, you copy
the code, open a browser to http://microsoft.com/devicelogin and paste
it in. Then follow the interactive authentication flow.</p>

<b>Azure CLI authentication</b> requires you to logon (in the notebook or 
a terminal) before authenticating to Microsoft Sentinel
<pre>az login</pre>
    
<p>You can change the authentication option used when calling "connect"
with the following.<br>
To force <b>Device authentication</b> add the following parameter
to the connect call
<pre>
qry_prov.connect(ws_config, mp_az_auth=False)
</pre>
</p>
<p>
To use <b>Azure CLI authentication</b>:
<pre>
qry_prov.connect(ws_config, mp_az_auth=["cli"])
</pre>
</p>
</details>


<details>
    <summary>Using WorkspaceConfig</summary>
Loading WorkspaceConfig with no parameters will use the details
of your "Default" workspace (see the Configuring Microsoft Sentinel settings section earlier)<br>

If you want to connect to a specific workspace use this syntax:<br>
   <pre>ws_config = WorkspaceConfig(workspace="WorkspaceName")</pre>
'WorkspaceName' should be one of the workspaces defined in msticpyconfig.yaml
</details>

In [ ]:
# Get the default Microsoft Sentinel workspace details from msticpyconfig.yaml

ws_config = msticpy.WorkspaceConfig()

# Connect to Microsoft Sentinel with our QueryProvider and config details
qry_prov.connect(ws_config)

## 6 Test your connection using a MSTICPy built-in Microsoft Sentinel query

To explore queries in more detail see the **A Tour of CyberSec Notebook Features** notebook.


In [ ]:
# The time parameters are taken from the qry_prov.query_prov time settings
# attribute, which provides the default query time range. You can
# change interactively this by running qry_prov.query_time.
alerts_df = qry_prov.SecurityAlert.list_alerts(start=qry_prov.query_time.start)

if alerts_df.empty:
    print("The query returned no rows for this time range. You might want to increase the time range")

# display first 5 rows of any results
alerts_df.head() # If you have no data you will just see the column headings displayed